<a href="https://colab.research.google.com/github/Anshuman-37/Netflix_prize_problem/blob/main/Netflix_Prize_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download -d netflix-inc/netflix-prize-data

 98% 669M/683M [00:05<00:00, 150MB/s]
100% 683M/683M [00:05<00:00, 126MB/s]


In [9]:
! unzip netflix-prize-data.zip


Archive:  netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          


In [16]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib 
matplotlib.use('nbagg')

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning' : 0})

import seaborn as sns
sns.set_style('whitegrid')

import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from 